In [10]:
import tensorflow as tf
#查看变量在哪个设备上
def run_with_location_trace(sess,op):
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    sess.run(op,options=run_options,run_metadata=run_metadata)
    for device in run_metadata.step_stats.dev_stats:
        print(device.device)
        for node in device.node_stats:
            print('  ',node.node_name)
#定义集群服务器
tasks = ['localhost:10003','localhost:10004']
jobs = {'local':tasks}
#定义集群变量
cluster = tf.train.ClusterSpec(jobs)
#定义分布服务器
server1 = tf.train.Server(cluster,job_name='local',task_index=0)
server2 = tf.train.Server(cluster,job_name='local',task_index=1)

#重置tf的图
tf.reset_default_graph()#把之前的删除
#定义变量
var = tf.Variable(initial_value=0.0,name='var')
#定义两个session会话
sess1 = tf.Session(server1.target)
sess2 = tf.Session(server2.target)
print(id(sess1.graph))
print(id(sess2.graph))
#只需要初始化一个会话中的即可
sess1.run(tf.global_variables_initializer())
print('Initial value of var in sess 1:',sess1.run(var))
print('Initial value of var in sess 2:',sess2.run(var))

sess2.run(var.assign_add(7.0))
print('value of var in sess1:',sess1.run(var))
print('value of var in sess2:',sess2.run(var))
run_with_location_trace(sess1,var)
#run_with_location_trace(sess2,var)
sess1.close()
sess2.close()

2613694600920
2613694600920
Initial value of var in sess 1: 0.0
Initial value of var in sess 2: 0.0
value of var in sess1: 7.0
value of var in sess2: 7.0
/job:local/replica:0/task:0/device:CPU:0
   _SOURCE
   var
/job:local/replica:0/task:1/device:CPU:0
   RecvTensor
/job:local/replica:0/task:0/device:CPU:0
   _SOURCE
   var
/job:local/replica:0/task:1/device:CPU:0
   _SOURCE
